In [ ]:
%pip install turicreate

In [62]:
import json
import pandas as pd
import glob
import os
import turicreate as tc
import shutil
import numpy as np

#os.remove('/Users/marcoincerti/Desktop/dataML/Data_ready.csv')

shutil.rmtree(r'/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/dataML/train')
shutil.rmtree(r'/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/dataML/test')

In [63]:
def normalize_data(data):
    prev_relative = data[0]['altitude']['relativeAltitude']
    data[0]['altitude']['relativeAltitude'] = 0.0
    data[0]['altitude']['differenceAltitude'] = 0.0
    
    prev_pressure = data[0]['altitude']['pressure']
    #data[0]['altitude']['pressure'] = 0.0
    data[0]['altitude']['differencePressure'] = 0.0
    
    for d in range(1, len(data)):
        tmp = data[d]['altitude']['relativeAltitude']
        tmp_pressure = data[d]['altitude']['pressure']
        if tmp - prev_relative == 0:
            #data[d]['altitude']['relativeAltitude'] = data[d -1]['altitude']['relativeAltitude']
            data[d]['altitude']['differenceAltitude'] = data[d -1]['altitude']['differenceAltitude']
        else:
            data[d]['altitude']['differenceAltitude'] = float(tmp - prev_relative)
            
        if tmp_pressure - prev_pressure == 0:
            data[d]['altitude']['differencePressure'] = data[d - 1]['altitude']['differencePressure']
        else:
            data[d]['altitude']['differencePressure'] = float(tmp_pressure - prev_pressure)
        prev_relative = tmp
        prev_pressure = tmp_pressure
        #tmp = data[d]['altitude']
    return data

In [64]:
from numpy import double


i = 0
df_list = []
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/stay/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)

    data = data['motionRecords']
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
       
    if len(data) > 10:
      data_norm = normalize_data(data)
      df = pd.json_normalize(data_norm, sep='_') 
      df['sessionId'] = i
      i +=1
      df['activity'] = "stay"
      df_list.append(df)
 
i = 0       
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/climb/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
    
    data = data['motionRecords']
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
    
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] == 0:
       data.pop(0)
       
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "stairs_up"
        df_list.append(df)
    
i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/descend/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
    
    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
       
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] == 0:
       data.pop(0)
        
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "stairs_down"
        df_list.append(df)
    


i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/elevator_up/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
 
    # for reading nested data [0] represents
    # the index value of the list
    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] == 0:
       data.pop(0)
        
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "elevator_up"
        df_list.append(df)
    

i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/elevator_down/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
 
    # for reading nested data [0] represents
    # the index value of the list
    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] == 0:
       data.pop(0)
        
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "eleveator_down"
        df_list.append(df)
    

df_final = pd.concat(df_list)
print(df_final.count())
df_final = df_final.dropna()
#df_final['altitude_relativeAltitude'].mask(df_final['altitude_relativeAltitude'] < 0.001, 0, inplace=True)
df_final['altitude_relativeAltitude'] = df_final['altitude_relativeAltitude'].replace([0],double(0.0001))
df_final['altitude_differenceAltitude'] = df_final['altitude_differenceAltitude'].replace([0],float(0.0001))
df_final['altitude_differencePressure'] = df_final['altitude_differencePressure'].replace([0],float(0.0001))
#df_final.drop(df_final[df_final.altitude_relativeAltitude == 0].index, inplace=True)
print(len(df_final))
df_final.head(200)

recordTimestamp                 189400
attitude_roll                   189400
attitude_pitch                  189400
attitude_yaw                    189400
altitude_pressure               189400
altitude_relativeAltitude       189400
altitude_differenceAltitude     189400
altitude_differencePressure     189400
rotationRate_x                  189400
rotationRate_y                  189400
rotationRate_z                  189400
accelerationWithOutGravity_x    189400
accelerationWithOutGravity_y    189400
accelerationWithOutGravity_z    189400
sessionId                       189400
activity                        189400
dtype: int64
189400


,recordTimestamp,attitude_roll,attitude_pitch,attitude_yaw,altitude_pressure,altitude_relativeAltitude,altitude_differenceAltitude,altitude_differencePressure,rotationRate_x,rotationRate_y,rotationRate_z,accelerationWithOutGravity_x,accelerationWithOutGravity_y,accelerationWithOutGravity_z,sessionId,activity
0,1.665046e+09,0.000506,0.208624,0.003961,101.325981,0.000100,0.000100,0.000100,0.438124,-0.276858,-0.458519,-0.251960,0.094276,0.053176,0,stay
1,1.665046e+09,-0.002514,0.217140,-0.007267,101.325981,0.000100,0.000100,0.000100,0.420521,-0.263474,-0.656042,-0.316332,0.094603,0.002760,0,stay
2,1.665046e+09,-0.003860,0.225396,-0.022842,101.325981,0.000100,0.000100,0.000100,0.396674,-0.224215,-0.865915,-0.326102,0.106732,-0.003314,0,stay
3,1.665046e+09,-0.003064,0.232822,-0.041989,101.325981,0.000100,0.000100,0.000100,0.335956,-0.113905,-0.986604,-0.263514,0.132549,0.029671,0,stay
4,1.665046e+09,0.000677,0.238846,-0.062169,101.325981,0.000100,0.000100,0.000100,0.271323,0.014296,-0.973841,-0.181154,0.174600,0.065133,0,stay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.665046e+09,-0.022236,0.169654,-0.813015,101.326248,-0.026423,-0.010569,0.000137,-0.892137,0.789671,-0.789898,-0.093762,-0.096570,-0.331052,0,stay
196,1.665046e+09,-0.005622,0.152377,-0.829024,101.326248,-0.026423,-0.010569,0.000137,-0.843645,0.616879,-0.762163,-0.101468,-0.154578,-0.279059,0,stay
197,1.665046e+09,0.006593,0.137685,-0.843301,101.326248,-0.026423,-0.010569,0.000137,-0.597325,0.412705,-0.650070,-0.091400,-0.156679,-0.305033,0,stay
198,1.665046e+09,0.015346,0.129340,-0.854912,101.326248,-0.026423,-0.010569,0.000137,-0.221398,0.326849,-0.511355,-0.094548,-0.138919,-0.351228,0,stay


In [65]:
# Column name mappings
cols = {
    #"recordTimestamp": "timestamp",
    #"attitude_roll": "attitude_roll",
    #"attitude_pitch": "attitude_pitch",
    #"attitude_yaw": "attitude_yaw",
    "altitude_pressure": "altitude_pressure",
    "altitude_relativeAltitude": "relativeAltitude",
    "rotationRate_x": "rotationRate_x",
    "rotationRate_y": "rotationRate_y",
    "rotationRate_z": "rotationRate_z",
    "accelerationWithOutGravity_x": "acceleration_x",
    "accelerationWithOutGravity_y": "acceleration_y",
    "accelerationWithOutGravity_z": "acceleration_z",
    "sessionId": "session_id",
    "activity": "activity",
    "altitude_differenceAltitude" : "differenceAltitude",
    'altitude_differencePressure' : 'differencePressure'
    }

csv_cols = ["rotationRate_x", "rotationRate_y", "rotationRate_z", "acceleration_x", "acceleration_y",
            "acceleration_z","differenceAltitude", 'differencePressure' ]

# Load csv data and rename columns
sf = tc.SFrame(data=df_final[list(cols.keys())]).rename(cols)

# Remove missing activies
sf = sf[sf['activity'] != '']
acts = sf['activity'].unique()

# Split data into training and testing sets
#train, test = sf.random_split(0.8)
train, test = tc.activity_classifier.util.random_split_by_session(sf, session_id='session_id', fraction=0.8)

# Write out training data
path = "train/"
os.mkdir(path)
for a in acts:
  # Check for folder
  cls_path = path + a + "/"
  if not os.path.exists(cls_path):
    os.mkdir(cls_path)
# Split data by activity & session and write to file
  sf_act = train[train['activity'] == a]
  for s in sf_act['session_id'].unique():
    sf_act[sf_act['session_id'] == s][csv_cols].save(cls_path + str(a)+ "_" + str(s) + ".csv")
# Write out testing data
path = "test/"
os.mkdir(path)
for a in acts:
  # Check for folder
  cls_path = path + a + "/"
  if not os.path.exists(cls_path):
    os.mkdir(cls_path)
# Split data by activity & session and write to file
  sf_act = test[test['activity'] == a]
  for s in sf_act['session_id'].unique():
    sf_act[sf_act['session_id'] == s][csv_cols].save(cls_path + str(a) + "_" + str(s) + ".csv")
